In [1]:
import pandas as pd
import sys
import os
import string
import numpy as np
import scipy.stats as stats
import re

In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):    
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [3]:
def calc_mean(list):
    if len(list) == 1:
        return list[0]
    else:    
        mean = np.mean(list)
        std = np.std(list)
        normal_list = []
        for l in list:
            z = (l-mean)/std
            if z <= 0.8 and z>= -1:
                normal_list.append(l)
        avg_value = np.mean(normal_list) # /1000 convert milisecond to second   
        return avg_value 

In [18]:
def results():
    root_path = '../archive'
    write_results = [f'{root_path}/read_write/CPP_R13_Experiment1_Write_times.dat', 
                     f'{root_path}/read_write/Java_R1_Experiment1_Write_times.dat',
                     f'{root_path}/read_write/Java_R2_Experiment1_Write_times.dat',
                     f'{root_path}/read_write/Java_R3_Experiment1_Write_times.dat']
   
    load_to_memory_results = [f'{root_path}/read_write/CPP_R13_Experiment1_LoadToMemory_times.dat',
                              f'{root_path}/read_write/Java_R1_Experiment1_LoadToMemory_times.dat',
                              f'{root_path}/read_write/Java_R2_Experiment1_LoadToMemory_times.dat',
                              f'{root_path}/read_write/Java_R3_Experiment1_LoadToMemory_times.dat']
   
    write_header = ["baseline","language","taskset","execution","platform","seq_rand","nrow","time"]
   
    load_to_memory_header = ["language","platform","taskset","nrow","time"]

    df_write = pd.DataFrame(columns=write_header)
    df_load_to_memory = pd.DataFrame(columns=load_to_memory_header)

    base_index = 10
    for path in write_results:        
        data = pd.read_csv( path, sep=',')
        df_index = [range(base_index, base_index + len(data), 1)]
        base_index += len(data)
        data.index=df_index
        print(data)


   

In [19]:
results()

         baseline language  taskset execution  platform    seq_rand      nrow  \
0    HandcodedCPP      CPP    False       CPU    Single  Sequential  10000000   
1    HandcodedCPP      CPP     True       CPU    Single  Sequential  10000000   
2    HandcodedCPP      CPP    False       CPU  Parallel  Sequential  10000000   
3    HandcodedCPP      CPP    False     Total    Single  Sequential  10000000   
4    HandcodedCPP      CPP     True     Total    Single  Sequential  10000000   
..            ...      ...      ...       ...       ...         ...       ...   
109  HandcodedCPP      CPP     True       CPU    Single  Sequential  10000000   
110  HandcodedCPP      CPP    False       CPU  Parallel  Sequential  10000000   
111  HandcodedCPP      CPP    False     Total    Single  Sequential  10000000   
112  HandcodedCPP      CPP     True     Total    Single  Sequential  10000000   
113  HandcodedCPP      CPP    False     Total  Parallel  Sequential  10000000   

       time  
0    137984  